
* [SystemML Compilation Chain](#SystemMLCompilationChain)
* [Basic HOP DAG Compilation](#BasicHOPDAGCompilation)
* [Static and Dynamic Rewrites](#StaticandDynamicRewrites)
	* [Examples Static Rewrites](#ExamplesStaticRewrites)
	* [Examples Dynamic Rewrites](#ExamplesDynamicRewrites)
	* [Rewrites and Operator Selection in Action](#RewritesandOperatorSelectioninAction)
* [Fused Operators](#FusedOperators)
* [Dynamic Recompilation](#DynamicRecompilation)


# SystemML Compilation Chain <a id="SystemMLCompilationChain"></a>

![SystemML Compilation Chain](images/Compilation-Chain1.png)

### Install Python interface for Graphviz

In [ ]:
!pip install graphviz

In [ ]:
!pip show graphviz

In [1]:
from systemml import MLContext, dml, getHopDAG, dmlFromResource, dmlFromUrl
from graphviz import Source
ml = MLContext(sc)

print "SystemML Version:", ml.version()
print "SystemML Built-Time:", ml.buildTime()

SystemML Version: 1.0.0-SNAPSHOT
SystemML Built-Time: 2017-08-14 01:28:05 UTC


# Basic HOP DAG Compilation <a id="BasicHOPDAGCompilation"></a>

Shape of the node (HOP's ExecType):
- CP: Ellipse
- SPARK: Box
- GPU: Trapezium
- MR: Parallelogram

Color of the node:
- Persistent/Transient Write/Read: Wheat
- AggBinaryOp: Orange
- BinaryOp: Blue
- ReorgOp: Green
- UnaryOp: Yellow

Hover over the node to find more details about the HOP (such as dimensions, number of non-zeros and memory estimates).

In [ ]:
prog = """
m = 10000; n = 3000; k = 500
X = matrix(seq(1, m*n), rows=m, cols=n)   # matrix constructor
Y = matrix(seq(n*k, 1), rows=n, cols=k)   # matrix constructor
Z = t(X %*% Y)                            # matrix multiplication followed by a transpose
"""
Source(getHopDAG(ml, dml(prog).output("Z")))

# Static and Dynamic Rewrites <a id="StaticandDynamicRewrites"></a>

- Static: size-independent rewrites
- Dynamic: size-dependent rewrites

## Examples Static Rewrites <a id="ExamplesStaticRewrites"></a>

- Common Subexpression Elimination
- Constant Folding
- Static Algebraic Simplification Rewrites
- Branch Removal
- Checkpoint injection (caching)
- Repartition injection

### Example Static Simplification Rewrites (size-independent patterns)

Simplify Operation over Matrix Multiplication: 
```
trace(X%*%Y) -> sum(X*t(Y))
```

In [ ]:
prog = """
m = 100000; n = 6000; k = 50000
X = rand(rows=m, cols=n)         # generate random matrix: X
Y = rand(rows=n, cols=k)         # generate random matrix: Y
Z = trace(X %*% Y)               # compute trace of X %*% Y
"""
Source(getHopDAG(ml, dml(prog).output("Z")))

The HOP DAG for the same script without rewrites:

In [ ]:
Source(getHopDAG(ml, dml(prog).output("Z"), apply_rewrites=False))

##  Examples Dynamic Rewrites <a id="ExamplesDynamicRewrites"></a>

- Dynamic Algebraic Simplification Rewrites
- Matrix Multiplication Chain Optimization


### Example Dynamic Simplification Rewrites (size-dependent patterns)

```
diag(X)%*%Y -> Y*X        iff ncol(X)=1 and ncol(Y)>1
```

In [ ]:
prog = """
m = 10000; k = 50000
X = rand(rows=m, cols=1)         # generate random vector: X
Y = rand(rows=m, cols=k)         # generate random matrix: Y
Z = diag(X) %*% Y                # construct diagonal of vector X and multiply it by Y
"""
Source(getHopDAG(ml, dml(prog).output("Z")))

The HOP DAG for the same script without rewrites:

In [ ]:
Source(getHopDAG(ml, dml(prog).output("Z"), apply_rewrites=False))

###  Matrix Multiplication Chain Optimization

![Matrix Multiplication Chain Optimization](images/MMChainOptimization1.png)

<span style="color:red">**Exercise:**</span> What's happening as we change dimensions of M1, M2, M3, M4 and M5 ?

In [ ]:
prog = """
dim1 = 1000; dim2 = 1000; dim3 = 1000; dim4 = 1
M1 = rand(rows=dim1, cols=dim2)                         # generate random matrix M1
M2 = matrix(seq(1,dim2*dim3), rows=dim2, cols=dim3)     # generate matrix M2
M3 = matrix(seq(dim3*dim4, 1), rows=dim3, cols=dim4)    # generate matrix M3
Z = (M1 %*% M2) %*% M3                                  # multiply matrix M1, M2 and M3

# Exercise:
# dim5 = 100; dim6 = 500;
# M4 = rand(rows=dim4, cols=dim5)
# M5 = matrix(1, rows=dim5, cols=dim6)
# Z = M1 %*% M2 %*% M3 %*% M4 %*% M5
"""
Source(getHopDAG(ml, dml(prog).output("Z")))

The HOP DAG for the same script without rewrites:

In [ ]:
Source(getHopDAG(ml, dml(prog).output("Z"), apply_rewrites=False))

## Rewrites and Operator Selection in Action <a id="RewritesandOperatorSelectioninAction"></a>	

Example: Use case Multi Logistic Regression, X: [10^8 x 10^3], K=1 (2 classes), 2GB mem

Please see [MultiLogReg.dml](https://github.com/apache/systemml/blob/master/scripts/algorithms/MultiLogReg.dml#L207-L208) for entire DML code.

- Original DML snippet of inner loop:

```
Q = P [, 1:K] * (X %*% ssX_V);
HV = t(X) %*% (Q - P [, 1:K] * (rowSums (Q) %*% matrix (1, rows = 1, cols = K)));
```

In [ ]:
# Get HOP DAG for Multi Logistic Regression (line: 207, 208)
# script = dmlFromUrl("https://raw.githubusercontent.com/apache/systemml/master/scripts/algorithms/MultiLogReg.dml")
script = dmlFromResource("scripts/algorithms/MultiLogReg.dml")
print('\n'.join(script.getScriptString().split('\n')[206:208]))

### Applied Rewrites

- After remove unnecessary (1) matrix multiply (2) unary aggregate: `rowSums(Q) %*% matrix(1, rows=1, cols=K)) -> Q`

```
Q = P[, 1:K] * (X %*% ssX_V);
HV = t(X) %*% (Q - P[, 1:K] * Q);
```

- After simplify distributive binary operation: `(Q - P[, 1:K] * Q) -> ((1 - P[, 1:K]) * Q)`

```
Q = P[, 1:K] * (X %*% ssX_V);
HV = t(X) %*% ((1 - P[, 1:K]) * Q);
```

- After simplify bushy binary operation: Expand `Q` in the second line and simplify.

```
HV = t(X) %*% (((1 - P[, 1:K]) * P[, 1:K]) * (X %*% ssX_V));
```

Additionally, after fusing binary dag to unary operation (sample proportion), we can compile an MapMMChain Lop (not shown in the below figure) where `w = P[, 1:K]`.

In [ ]:
# Generate data for running Multi Logistic Regression
dataGenScript = """
X = rand(rows=10^8, cols=10^3)                # generate random feature matrix X 
Y_vec = rand(rows=10^8, cols=1) > 0           # generate random label vector Y_vec
"""
X, Y_vec = ml.execute(dml(dataGenScript).output("X", "Y_vec")).get("X", "Y_vec")

script.input(X=X, Y_vec=Y_vec).output("B_out")
script.input("$X", " ").input("$Y", " ").input("$B", " ")
from pyspark.conf import SparkConf
conf = SparkConf().set("spark.driver.memory", "20g").set("spark.executor.memory", "20g")
Source(getHopDAG(MLContext(sc), script, conf=conf, lines=[209, 210]))

# Fused Operators <a id="FusedOperators"></a>	

![Fused WS Loss Operator](images/FusedOpWSLoss.png)

# Dynamic Recompilation <a id="DynamicRecompilation"></a>	

The above techniques work very well if we can infer intermediate sizes and sparsity, which
is true for simple iterative algorithms where the training data is accessed read-only and thus all important
operations are known. For other types of programs, the sizes or sparsity of intermediates may be unknown
during initial compilation. We use dynamic recompilation as our robust fallback strategy for these cases.

**Example ML Program Scenarios**:
- Scripts with complex function call patterns
- Scripts with UDFs
- Data-dependent operators
- Computed size expressions
- Changing dimensions or sparsity

In [ ]:
prog = """
X = rand(rows=100000, cols=100)      # generate random matrix X
Y = rand(rows=100000, cols=1)        # generate random vector Y
Y = table( seq(1,nrow(X)), Y )       # perform data-dependent operation: table
grad = t(X) %*% Y                    # transpose X and multiply it with Y
"""
Source(getHopDAG(ml, dml(prog).output("grad"), with_subgraph=True))